<a href="https://colab.research.google.com/github/meliusa/Utilizing-Decision-Tree-Method-for-Big-Data-Analysis-with-Spark-on-Google-Colab/blob/main/Utilizing_Decision_Tree_Method_for_Big_Data_Analysis_with_Spark_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TI-3G :
- Meliusa Nora Hariyanti
- Dherisma Hanindita Utami



install library pyspark ke google colab

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=f4ba0ef6241ae035135c9316f1f23efbe4ee751afa8b6612b3a87979d6fafda6
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


import library SparkSession dan DecisionTreeClassifier:

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import DecisionTreeClassifier

buat sesi SparkSession:

In [ ]:
spark = SparkSession.builder.appName("DecisionTreeExample").getOrCreate()

load dataset

In [ ]:
dataset = spark.read.format("csv").option("header", "true").load("data.csv")

pemrosesan dan persiapan data
1. jika diperlukan, lakukan pemrosesan data seperti pembersihan, pengubahan format, dan pemilihan fitur yang relevan.
2. pisahkan data menjadi data pelatihan (train) dan data pengujian (test):

In [ ]:
(trainData, testData) = dataset.randomSplit([0.7, 0.3])

definisikan model Decision Tree:

In [ ]:
dt = DecisionTreeClassifier(labelCol="quality", featuresCol="fixed acidity")

kita convert dulu type variabelnya ke float agar bisa di proses

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

In [ ]:
assembler = VectorAssembler(inputCols=trainData.columns[:-1], outputCol="last1")


In [ ]:
trainData = trainData.withColumn("volatile acidity", trainData["volatile acidity"].cast("float"))
trainData = trainData.withColumn("citric acid", trainData["citric acid"].cast("float"))
trainData = trainData.withColumn("residual sugar", trainData["residual sugar"].cast("float"))
trainData = trainData.withColumn("chlorides", trainData["chlorides"].cast("float"))
trainData = trainData.withColumn("free sulfur dioxide", trainData["free sulfur dioxide"].cast("float"))
trainData = trainData.withColumn("total sulfur dioxide", trainData["total sulfur dioxide"].cast("float"))
trainData = trainData.withColumn("density", trainData["density"].cast("float"))
trainData = trainData.withColumn("pH", trainData["pH"].cast("float"))
trainData = trainData.withColumn("sulphates", trainData["sulphates"].cast("float"))
trainData = trainData.withColumn("alcohol", trainData["alcohol"].cast("float"))
trainData = trainData.withColumn("quality", trainData["quality"].cast("float"))


transformasikan DataFrame "trainData" menggunakan VectorAssembler:

In [ ]:
trainData = assembler.transform(trainData)

latih model menggunakan data pelatihan:

In [ ]:
model = dt.fit(trainData)

lakukan prediksi menggunakan data pengujian:

In [ ]:
predictions = model.transform(trainData)

evaluasi performa model, misalnya dengan menggunakan metrik evaluasi seperti akurasi, presisi, atau recall:

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="quality", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)


Accuracy: 0.6518650088809946
